# Import Figure8 labels to Custom Vision Service

In [ ]:
# install packages if needed
import sys
!{sys.executable} -m pip install azure-cognitiveservices-vision-customvision
!{sys.executable} -m pip install azureml-sdk
!{sys.executable} -m pip install azureml-contrib-dataset
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install Pillow

In [1]:
import json, os, shutil, requests, math
import pandas as pd

from PIL import Image
from io import BytesIO

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
# azureml-contrib-dataset of version 1.0.72 or higher is required

## 1. Set up Custom Vision project

Enter the details for your Custom Vision endpoint and training key below:

In [2]:
# ENDPOINT = 'https://YOUR_REGION.api.cognitive.microsoft.com'
# training_key = "<CUSTOM VISION TRAINING KEY>"

ENDPOINT = 'https://westus2.api.cognitive.microsoft.com'
training_key = "bc43db20938e453b89b5fffc9ccbc8f9"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint=ENDPOINT, credentials=credentials)

### Option 1: Create new project

The below cell will create a new Custom Vision project.  Enter a name for your project below:

Note: If you have an existing project, skip to the next cell.

In [3]:
# project_name = "<PROJECT NAME>"
project_name = "test2"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print("Creating new project...")
project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)
print(project.name, "project created")

Creating new project...
test2 project created


### Option 2: Update existing Custom Vision project

The below cell will get an existing project by ID.  Enter your project ID below (you can retrieve this ID from your project in the [Custom Vision portal](http://customvision.ai)):

In [ ]:
project_id = "<PROJECT ID>"

# Get existing project
project = trainer.get_project(project_id = project_id) 
print(project.name, "project retrieved")

## 2. Read labeled dataset from csv

In [4]:
# filename = "<PATH TO CSV FILE WITH LABELS>" 
filename = "../data/mantech_final_subset.csv" 

df = pd.read_csv(filename)
df = df[['annotation','image_url','image_name']]
df.head()

,annotation,image_url,image_name
0,"[{""id"": ""1322953c-2209-4113-8b51-b2a2d2311e69""...",https://cf-315f62h.s3.us-east-1.amazonaws.com/...,SatImg (13).jpg
1,"[{""id"": ""1e8fa8b9-e530-4b45-bd56-eb30f5dfc9ef""...",https://cf-315f62h.s3.us-east-1.amazonaws.com/...,SatImg (13).jpg
2,"[{""id"": ""bf99ed99-f008-46a0-b0ef-3588e329dd91""...",https://cf-315f62h.s3.us-east-1.amazonaws.com/...,SatImg (13).jpg
3,"[{""id"": ""34b9ff40-bfd7-47a0-9fd3-e6f9c72928f6""...",https://cf-315f62h.s3.us-east-1.amazonaws.com/...,SatImg (13).jpg
4,"[{""id"": ""521e21d4-99a2-44ad-abb1-3c4aa4d31bfb""...",https://cf-315f62h.s3.us-east-1.amazonaws.com/...,SatImg (13).jpg


### Prep images and format tags for Custom Vision

In [5]:
def get_image_from_url(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    width = img.size[0]
    height = img.size[1]
    img_bytes = image_to_bytes(img)
    return img_bytes, width, height

def get_bbox(label, width, height):
    l = label['x']/width
    t = label['y']/height
    w = label['w']/width
    h = label['h']/height  
    return l,t,w,h

def image_to_bytes(image):
    # Convert image to byte stream
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    return img_byte_arr


In [6]:
tagged_ims = []
tags = trainer.get_tags(project.id)

num_labels = df.shape[0]

previous_url = ''
regions = []
for i, row in df.iterrows():
    
    # handle first as edge case to get pointer to previous image url
    if i == 0:
        previous_url = row['image_url']
        img_bytes, width, height = get_image_from_url(previous_url)
        filename = row['image_name']
        print(f"Processing image: {filename}, label {i}/{num_labels}")
        
    # get image url and label
    img_url = row['image_url']
    label = json.loads(row['annotation'])
    
    # retrieve tag object by label name
    label_name = 'vehicle'
    try:
        index = [x.name for x in tags].index(label_name)
        tag = tags[index]
    # create tag if it does not exist yet
    except:
        print("Creating new tag for:", label_name)
        tag = trainer.create_tag(project.id, label_name)
        tags = trainer.get_tags(project.id)
      
    # create bounding box regions
    coords = label[0]['coordinates']   
    l,t,w,h = get_bbox(coords, width, height)
    regions.append(Region(tag_id=tag.id,left=l,top=t,width=w,height=h))
    
    # when we get to a new image
    if img_url != previous_url:
        
        # create image file entry for all data from previous image and append to image set
        tagged_ims.append(ImageFileCreateEntry(name=filename, contents=img_bytes, regions=regions))
    
        regions = []
        
        # get info from new image
        img_bytes, width, height = get_image_from_url(img_url)
        filename = row['image_name']

        print(f"Processing image: {filename}")
        
    # update previous url to current
    previous_url = img_url

Processing image: SatImg  (13).jpg
Creating new tag for: vehicle
Processing image: SatImg  (19).jpg
Processing image: SatImg  (24).jpg
Processing image: SatImg  (26).jpg
Processing image: SatImg  (27).jpg
Processing image: SatImg  (28).jpg
Processing image: SatImg  (29).jpg
Processing image: SatImg  (30).jpg
Processing image: SatImg  (50).jpg
Processing image: SatImg  (56).jpg
Processing image: SatImg  (57).jpg
Processing image: SatImg  (58).jpg
Processing image: SatImg  (59).jpg
Processing image: SatImg  (80).jpg
Processing image: SatImg  (86).jpg
Processing image: SatImg  (87).jpg
Processing image: SatImg  (88).jpg
Processing image: SatImg  (89).jpg
Processing image: SatImg  (117).jpg
Processing image: SatImg  (181).jpg
Processing image: SatImg  (251).jpg
Processing image: SatImg  (252).jpg
Processing image: SatImg  (273).jpg
Processing image: SatImg  (274).jpg
Processing image: SatImg  (292).jpg
Processing image: SatImg  (293).jpg
Processing image: SatImg  (294).jpg
Processing image

Processing image: SatImg  (154).jpg
Processing image: SatImg  (155).jpg
Processing image: SatImg  (156).jpg
Processing image: SatImg  (166).jpg
Processing image: SatImg  (168).jpg
Processing image: SatImg  (169).jpg
Processing image: SatImg  (170).jpg
Processing image: SatImg  (171).jpg
Processing image: SatImg  (177).jpg
Processing image: SatImg  (178).jpg
Processing image: SatImg  (179).jpg
Processing image: SatImg  (180).jpg
Processing image: SatImg  (190).jpg
Processing image: SatImg  (191).jpg
Processing image: SatImg  (192).jpg
Processing image: SatImg  (193).jpg
Processing image: SatImg  (201).jpg
Processing image: SatImg  (202).jpg
Processing image: SatImg  (203).jpg
Processing image: SatImg  (204).jpg
Processing image: SatImg  (205).jpg
Processing image: SatImg  (206).jpg
Processing image: SatImg  (212).jpg
Processing image: SatImg  (216).jpg
Processing image: SatImg  (217).jpg
Processing image: SatImg  (218).jpg
Processing image: SatImg  (220).jpg
Processing image: SatImg  (2

Processing image: SatImg  (693).jpg
Processing image: SatImg  (694).jpg
Processing image: SatImg  (695).jpg
Processing image: SatImg  (696).jpg
Processing image: SatImg  (697).jpg
Processing image: SatImg  (699).jpg
Processing image: SatImg  (700).jpg
Processing image: SatImg  (701).jpg
Processing image: SatImg  (705).jpg
Processing image: SatImg  (706).jpg
Processing image: SatImg  (732).jpg
Processing image: SatImg  (733).jpg
Processing image: SatImg  (734).jpg
Processing image: SatImg  (735).jpg
Processing image: SatImg  (736).jpg
Processing image: SatImg  (737).jpg
Processing image: SatImg  (738).jpg
Processing image: SatImg  (739).jpg
Processing image: SatImg  (740).jpg
Processing image: SatImg  (741).jpg
Processing image: SatImg  (742).jpg
Processing image: SatImg  (743).jpg
Processing image: SatImg  (747).jpg
Processing image: SatImg  (759).jpg
Processing image: SatImg  (772).jpg
Processing image: SatImg  (773).jpg
Processing image: SatImg  (774).jpg
Processing image: SatImg  (7

Processing image: Titan  (60).jpg
Processing image: Titan  (61).jpg
Processing image: Titan  (62).jpg
Processing image: Titan  (63).jpg
Processing image: Titan  (64).jpg
Processing image: Titan  (66).jpg
Processing image: Titan  (69).jpg
Processing image: Titan  (70).jpg
Processing image: Titan  (75).jpg
Processing image: Titan  (77).jpg
Processing image: Titan  (85).jpg
Processing image: Titan  (86).jpg
Processing image: Titan  (87).jpg
Processing image: Titan  (93).jpg
Processing image: Titan  (98).jpg
Processing image: Titan  (101).jpg
Processing image: Titan  (102).jpg
Processing image: Titan  (103).jpg
Processing image: Titan  (106).jpg
Processing image: Titan  (107).jpg
Processing image: Titan  (110).jpg
Processing image: Titan  (111).jpg
Processing image: Titan  (112).jpg
Processing image: Titan  (113).jpg
Processing image: Titan  (114).jpg
Processing image: Titan  (115).jpg
Processing image: Titan  (116).jpg
Processing image: Titan  (117).jpg
Processing image: Titan  (118).jpg


Processing image: uav  (359).jpg
Processing image: uav  (362).jpg
Processing image: uav  (363).jpg
Processing image: uav  (364).jpg
Processing image: uav  (366).jpg
Processing image: uav  (372).jpg
Processing image: uav  (374).jpg
Processing image: uav  (391).jpg
Processing image: uav  (392).jpg
Processing image: uav  (393).jpg
Processing image: uav  (395).jpg
Processing image: uav  (396).jpg
Processing image: uav  (397).jpg
Processing image: uav  (399).jpg
Processing image: uav  (400).jpg
Processing image: uav  (401).jpg
Processing image: uav  (404).jpg
Processing image: uav  (405).jpg
Processing image: uav  (406).jpg
Processing image: uav  (407).jpg
Processing image: uav  (413).jpg
Processing image: uav  (435).jpg
Processing image: uav  (444).jpg
Processing image: uav  (456).jpg
Processing image: uav  (457).jpg
Processing image: uav  (458).jpg
Processing image: uav  (459).jpg
Processing image: uav  (465).jpg
Processing image: uav  (466).jpg
Processing image: uav  (467).jpg
Processing

Processing image: uav  (813).jpg
Processing image: uav  (815).jpg
Processing image: uav  (817).jpg
Processing image: uav  (818).jpg
Processing image: uav  (819).jpg
Processing image: uav  (820).jpg
Processing image: uav  (821).jpg
Processing image: uav  (822).jpg
Processing image: uav  (825).jpg
Processing image: uav  (827).jpg
Processing image: uav  (829).jpg
Processing image: uav  (830).jpg
Processing image: uav  (831).jpg
Processing image: uav  (832).jpg
Processing image: Titan  (82).jpg
Processing image: uav  (514).jpg
Processing image: uav  (687).jpg
Processing image: uav  (689).jpg
Processing image: uav  (745).jpg
Processing image: uav  (806).jpg
Processing image: uav  (343).jpg
Processing image: Titan  (105).jpg
Processing image: Titan  (109).jpg
Processing image: Titan  (127).jpg
Processing image: Titan  (175).jpg
Processing image: Titan  (176).jpg
Processing image: Titan  (186).jpg
Processing image: Titan  (188).jpg
Processing image: Titan  (195).jpg
Processing image: Titan  (

Processing image: SatImg  (827).jpg
Processing image: SatImg  (828).jpg
Processing image: SatImg  (844).jpg
Processing image: SatImg  (845).jpg
Processing image: SatImg  (849).jpg
Processing image: SatImg  (877).jpg
Processing image: SatImg  (879).jpg
Processing image: SatImg  (880).jpg
Processing image: SatImg  (881).jpg
Processing image: SatImg  (883).jpg
Processing image: SatImg  (885).jpg
Processing image: SatImg  (888).jpg
Processing image: SatImg  (898).jpg
Processing image: SatImg  (900).jpg
Processing image: SatImg  (901).jpg
Processing image: SatImg  (902).jpg
Processing image: SatImg  (904).jpg
Processing image: SatImg  (906).jpg
Processing image: SatImg  (914).jpg
Processing image: SatImg  (915).jpg
Processing image: SatImg  (916).jpg
Processing image: SatImg  (920).jpg
Processing image: SatImg  (923).jpg
Processing image: SatImg  (924).jpg
Processing image: SatImg  (935).jpg
Processing image: SatImg  (942).jpg
Processing image: SatImg  (943).jpg
Processing image: SatImg  (9

Processing image: SatImg  (552).jpg
Processing image: SatImg  (561).jpg
Processing image: SatImg  (564).jpg
Processing image: SatImg  (565).jpg
Processing image: SatImg  (567).jpg
Processing image: SatImg  (568).jpg
Processing image: SatImg  (576).jpg
Processing image: SatImg  (579).jpg
Processing image: SatImg  (580).jpg
Processing image: SatImg  (587).jpg
Processing image: SatImg  (588).jpg
Processing image: SatImg  (590).jpg
Processing image: SatImg  (592).jpg
Processing image: SatImg  (593).jpg
Processing image: SatImg  (594).jpg
Processing image: SatImg  (596).jpg
Processing image: SatImg  (600).jpg
Processing image: SatImg  (602).jpg
Processing image: SatImg  (603).jpg
Processing image: SatImg  (604).jpg
Processing image: SatImg  (605).jpg
Processing image: SatImg  (606).jpg
Processing image: SatImg  (608).jpg
Processing image: SatImg  (610).jpg
Processing image: SatImg  (611).jpg
Processing image: SatImg  (618).jpg
Processing image: SatImg  (619).jpg
Processing image: SatImg  (6

## 4. Upload images and tags to Custom Vision

In [7]:
len(tagged_ims)

1792

In [8]:
counter = 0

for i in range(0, len(tagged_ims), 64): 
    batch = tagged_ims[i:i+64]
    result = trainer.create_images_from_files(project.id, images=batch)

    counter += 1
    print(f"Uploaded batch {counter} of {math.ceil(len(tagged_ims)/64)}: {result.is_batch_successful}")
    
    if not result.is_batch_successful:
    print("Image batch upload failed.")
    for image in result.images:
        print("Image status: ", image.status)

Uploaded batch 1 of 28: True
Uploaded batch 2 of 28: False
Uploaded batch 3 of 28: True
Uploaded batch 4 of 28: True
Uploaded batch 5 of 28: True
Uploaded batch 6 of 28: True
Uploaded batch 7 of 28: True
Uploaded batch 8 of 28: True
Uploaded batch 9 of 28: True
Uploaded batch 10 of 28: True
Uploaded batch 11 of 28: True
Uploaded batch 12 of 28: True
Uploaded batch 13 of 28: True
Uploaded batch 14 of 28: True
Uploaded batch 15 of 28: True
Uploaded batch 16 of 28: True
Uploaded batch 17 of 28: True
Uploaded batch 18 of 28: True
Uploaded batch 19 of 28: True
Uploaded batch 20 of 28: True
Uploaded batch 21 of 28: True
Uploaded batch 22 of 28: True
Uploaded batch 23 of 28: True
Uploaded batch 24 of 28: True
Uploaded batch 25 of 28: True
Uploaded batch 26 of 28: True
Uploaded batch 27 of 28: True
Uploaded batch 28 of 28: True
